<a href="https://colab.research.google.com/github/nihima10110/Home-appliances/blob/main/Question_Answering_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Packages

In [ ]:
!pip install langchain                       #Main Freamework
!pip install langchain_community   #For loaders,tools and vecotrs
!pip install langchain_core             #Prompts,documents,runnable
!pip install langchain_ollama         #local LLM

Download and Pull Ollama

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Launch Ollama in Background

In [ ]:
import threading      #run function parallel in background
import subprocess  #contunue running
import time              #pause the program

def run_ollama_serve():
  subprocess.Popen(["ollama","serve"])

thread=threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)



Pulling the Embedding Model

In [ ]:
!pip install ollama

In [ ]:
!ollama pull nomic-embed-text
!ollama pull gemma3:4b

In [ ]:
!!pip install faiss-cpu             #Facebook AI Similarity Search

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
embedding_model=SentenceTransformer('all-MiniLM-L6-v2')
#use to convert text into vector
llm=ChatOllama(model="gemma3:4b")
#use for reolying the questins

In [ ]:
chunks= [
    "The Eiffel Tower is located in Paris.",
    "Python is a programming language used for data science.",
    "OpenAI develops large language models like ChatGPT.",
    "The Great Wall of China is visible from space.",
    "Water boils at 100 degrees Celsius under standard conditions.",
    "The mitochondrion is the powerhouse of the cell.",
    "Mount Everest is the tallest mountain above sea level.",
    "Shakespeare wrote Hamlet and Macbeth.",
    "Tesla produces electric vehicles and solar products.",
    "Amazon is a large online retailer founded by Jeff Bezos.",
    "The Moon orbits the Earth approximately every 27.3 days.",
    "The speed of light is about 299,792 kilometers per second.",
    "Albert Einstein developed the theory of relativity.",
    "Venus is the second planet from the Sun.",
    "The Pacific Ocean is the largest and deepest ocean on Earth.",
    "Bacteria are single-celled microorganisms.",
    "The capital of Japan is Tokyo.",
    "Photosynthesis occurs in the chloroplasts of plant cells.",
    "Leonardo da Vinci painted the Mona Lisa.",
    "Bitcoin is a decentralized digital currency."
]


In [ ]:
chunk_vectors=embedding_model.encode(chunks)
#convert chunks into vectors
index=faiss.IndexFlatL2(chunk_vectors.shape[1])
#indexing all as small or large distance by using L2(Eucledian) distance
index.add(np.array(chunk_vectors))
# adds all the chunk vectors to the FAISS index for searching them later

In [ ]:
prompt_template="""Use the following context to answer the questions:
Context: {context}
Question: {question}
Answer:"""

In [ ]:
prompt=PromptTemplate.from_template(prompt_template)
chain=LLMChain(llm=llm,prompt=prompt)

In [ ]:
questions = [
    "Where is the Eiffel Tower?",
    "Who painted the Mona Lisa?",
    "What does Tesla produce?",
    "What is the speed of light?",
    "Where does photosynthesis happen in plant cells?"
]

In [ ]:
result=[]

In [ ]:
for question in questions:
  question_vector=embedding_model.encode([question])
  #Convert questions into vector
  _,idx=index.search(np.array(question_vector), k=1)
  #searching the most similar vector in FAISS that match with question
  #k=1 means the first chunk
  #_ is used for ignoring because the index.search gives two strings
  #first one is distance and second one is indecies
  #as we only need indecies so ignore the distance by using _
  top_chunk=chunks[idx[0][0]]
  #to find the top most chunk
  #suppose chunk=[[2]] then idx[0]=[2] and idx[0][0]=2
  answer=chain.run(context=top_chunk,question=question).strip()
  #run the question and get the answer
  result.append((question,top_chunk,answer))



In [ ]:
for i, (question, context, answer) in enumerate(result, 1):
    print(f"🧠 Question {i}: {question}")
    print(f"📚 Retrieved Context: {context}")
    print(f"✅ Answer: {answer}")
    print("-" * 70)